In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.appName("adaptive_query_execution").getOrCreate()
spark.conf.set("spark.sql.adaptive.enabled", "false")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

# Join skew

In [ ]:
df_trans = spark.read.format("parquet").load("/opt/bitnami/spark/custom_data/chapter7/transactions/")
df_cust = spark.read.format("parquet").load("/opt/bitnami/spark/custom_data/chapter7/customers/")

In [ ]:
(
        df_trans.groupBy("cust_id")
        .agg(F.countDistinct("txn_id").alias("ct"))
        .orderBy(F.desc("ct"))
        .show(5, truncate=False)
    )

In [ ]:
df_txn_details = df_trans.join(df_cust, on="cust_id", how="inner")

In [ ]:
df_txn_details.count()

# Using AQE

In [ ]:
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewdedJoin.enabled", "true")

In [ ]:
df_txn_details = df_transactions.join(df_cust, on="cust_id", how="inner")

In [ ]:
df_txn_details.count()

In [ ]:
spark.stop()